# Exercices
With each exercice will teach you one aspect of deep learning. The process of machine learning can be decompose in 7 steps :

* Data preparation
* Model definition
* Model training
* Model evaluation
* Hyperparameter tuning
* Prediction

## 4 - Model evaluation

- 4.1 Visualize metrics during training (matplot)
- 4.2 Detect overfitting / underfitting
- 4.2 Validate model with unseen dataset
- 4.3 Visualize confusion matrix

### Load dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(images, labels):
    npimg = images.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title("Ground Truth: {}".format(labels))

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

data_path = './data'

#trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
trans = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

# if not exist, download mnist dataset
train_set = dset.MNIST(root=data_path, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=data_path, train=False, transform=trans, download=True)

batch = 4

data_train_loader = DataLoader(train_set, batch_size=batch, shuffle=True, num_workers=8)
data_test_loader = DataLoader(test_set, batch_size=batch, num_workers=8)

### Network architecture

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Load leNet & co.

In [4]:
leVieuxNet = Net()
optimizer = optim.SGD(leVieuxNet.parameters(), lr=0.01)

checkpoint = torch.load('checkpoint-MKTD-pytorch-3.last')
leVieuxNet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

### Compute performance on test set

In [5]:
def compute_accuracy(test_data, network):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data:
            images, labels = data
            outputs = network(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [6]:
acc = compute_accuracy(data_test_loader, leVieuxNet)
            
print('Accuracy of the network on the {} test images: {:.3f}%'.format(len(data_test_loader), acc))

Accuracy of the network on the 2500 test images: 96.870%


### Train a new net with another loss 

In [17]:
leNet = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(leNet.parameters(), lr=0.01)

In [18]:
def train(model, criterion, optimizer, train_data, max_epoch):

    for epoch in range(max_epoch):  # loop over the dataset multiple times
        model.train()

        running_loss = 0.0
        for i, (images, labels) in enumerate(train_data):
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images)

            # align vectors labels <=> outputs
#             label_vect = torch.zeros(outputs.shape[0], outputs.shape[1], dtype=torch.long)
#             for j in range(0, len(labels)):
#                 label_vect[j, labels[j]] = 1.0 
            
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

        print('[{:d}] loss: {:.5f}'.format(epoch + 1, running_loss / (batch*len(train_data))))

    print('Finished Training')

In [19]:
train(leNet, criterion, optimizer, data_train_loader, 3)

[1] loss: 0.09175
[2] loss: 0.01689
[3] loss: 0.01207
Finished Training


In [22]:
acc = compute_accuracy(data_test_loader, leNet)
            
print('Accuracy of the network on the {} test images: {:.3f}%'.format(len(data_test_loader), acc))

Accuracy of the network on the 2500 test images: 98.280%


## Testing another gradient descent variant

http://ruder.io/optimizing-gradient-descent/

In [24]:
leNet = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(leNet.parameters(), lr=2e-3)
train(leNet, criterion, optimizer, data_train_loader, 3)
acc = compute_accuracy(data_test_loader, leNet)
            
print('Accuracy of the network on the {} test images: {:.3f}%'.format(len(data_test_loader), acc))

[1] loss: 0.03603
[2] loss: 0.02040
[3] loss: 0.01881
Finished Training
Accuracy of the network on the 2500 test images: 98.090%


In [30]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in data_test_loader:
        images, labels = data
        outputs = leNet(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Precision of {:s}: {:2f}%'.format(classes[i], 100 * class_correct[i] / class_total[i]))

Precision of 0: 98.775510%
Precision of 1: 99.207048%
Precision of 2: 97.383721%
Precision of 3: 96.732673%
Precision of 4: 99.389002%
Precision of 5: 98.654709%
Precision of 6: 97.807933%
Precision of 7: 99.124514%
Precision of 8: 96.919918%
Precision of 9: 96.828543%


## TODO
- Visualize confusion matrix
- play with batch size, epoch
- Visualize metrics during training (matplot)
- Detect overfitting / underfitting
- stopping criteria